In [1]:
import numpy as np

import matplotlib.pyplot as plt

from PyQt6.QtWidgets import (
    QMainWindow, QApplication, QWidget,
    QLabel, QCheckBox, QComboBox,
    QPushButton, QTextEdit,
    QLineEdit, QSpinBox, QDoubleSpinBox, QSlider,
    QVBoxLayout, QHBoxLayout, QGridLayout, QFormLayout, QWIDGETSIZE_MAX
)
from PyQt6.QtCore import Qt, QSize, QPoint
from PyQt6.QtGui import QPixmap, QPalette, QColor, QImage

import numpy as np

from random import randint


In [2]:
def show_image(image, no_show=False):
    plt.imshow(image)
    xticks = plt.xticks()
    yticks = plt.yticks()

    plt.xticks([])
    plt.yticks([])

    if not no_show:
        plt.show()

In [3]:
from python_coreml_stable_diffusion.pipeline import *

/Users/kingcake/Venvs/neuro/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
args = argparse.Namespace(
    prompt='a photo of an astronaut riding a horse on mars', i='models/coreml-stable-diffusion-2-base_original_packages', o='output', seed=96, model_version='stabilityai/stable-diffusion-2-base', compute_unit='ALL', scheduler=None, num_inference_steps=50, guidance_scale=7.5, pickle=None
)

In [5]:
model = load_model(args)

INFO:python_coreml_stable_diffusion.pipeline:Initializing PyTorch pipe for reference configuration
Fetching 16 files: 100%|██████████| 16/16 [00:00<00:00, 4473.33it/s]
INFO:python_coreml_stable_diffusion.pipeline:Loading Core ML pipe
INFO:python_coreml_stable_diffusion.pipeline:Removed PyTorch pipe to reduce peak memory consumption
INFO:python_coreml_stable_diffusion.pipeline:Loading Core ML models in memory from models/coreml-stable-diffusion-2-base_original_packages
INFO:python_coreml_stable_diffusion.coreml_model:Loading text_encoder mlpackage
INFO:python_coreml_stable_diffusion.coreml_model:Loading models/coreml-stable-diffusion-2-base_original_packages/Stable_Diffusion_version_stabilityai_stable-diffusion-2-base_text_encoder.mlpackage


KeyboardInterrupt: 

In [ ]:
# def callback(i, t, latents):
#     # show the image
#     image = (model.decode_latents(latents)[0] * 255).astype(np.uint8)

#     qt_image = QImage(image.shape[1], image.shape[0], QImage.Format.Format_RGB888)
#     for x in range(image.shape[1]):
#         for y in range(image.shape[0]):
#             qt_image.setPixel(x, y, QColor(image[y, x, 0], image[y, x, 1], image[y, x, 2]).rgb())

#     show_image(image)

In [ ]:
# ret = model(
#     "a ((perfect)) circle",
#     seed=100,
#     num_inference_steps=2,
#     callback=callback
# )

In [7]:
from PyQt6.QtWidgets import (
    QMainWindow, QApplication, QWidget,
    QLabel, QCheckBox, QComboBox,
    QPushButton, QTextEdit,
    QLineEdit, QSpinBox, QDoubleSpinBox, QSlider,
    QVBoxLayout, QHBoxLayout, QGridLayout, QFormLayout, QProgressBar, QWIDGETSIZE_MAX
)
from PyQt6.QtCore import Qt, QSize, QPoint
from PyQt6.QtGui import QPixmap, QPalette, QColor, QImage

import numpy as np

from random import randint

from threading import Thread


def np_to_pixmap(image):
    # convert numpy array to QPixmap
    image = np.uint8(image * 255)
    qt_image = QImage(image.shape[1], image.shape[0], QImage.Format.Format_RGB888)
    for x in range(image.shape[1]):
        for y in range(image.shape[0]):
            qt_image.setPixel(x, y, QColor(image[y, x, 0], image[y, x, 1], image[y, x, 2]).rgb())
    
    return QPixmap.fromImage(qt_image)


class MainWindow(QMainWindow):

    def __init__(self, ui):
        super(MainWindow, self).__init__()
        self.ui = ui

        self.setWindowTitle("Biburator")

        grid = QGridLayout()

        widget = QWidget()
        widget.setLayout(grid)
        self.setCentralWidget(widget)

        pixmap = QPixmap(512, 512)
        pixmap.fill(QColor(215, 215, 215))
        self.image_label = QLabel("This is a label")
        self.image_label.setPixmap(pixmap)
        self.image_label.setBaseSize(pixmap.size())

        self.progress_bar = QProgressBar()

        self.prompt_input = QTextEdit()
        self.prompt_input.setPlaceholderText("Enter promt here")

        self.steps_input = QLineEdit("30")
        self.steps_input.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.steps_label = QLabel("Steps")

        self.guidence_input = QLineEdit("7.5")
        self.guidence_input.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.guidence_label = QLabel("Guidence")

        self.seed_input = QLineEdit("-1")
        self.seed_input.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.seed_label = QLabel("Seed")

        self.count_input = QLineEdit("1")
        self.count_input.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.count_label = QLabel("Count")

        self.generate_button = QPushButton("Generate")
        self.generate_button.clicked.connect(self.button_clicked)

        opts = [
            [
                self.steps_label,
                self.steps_input,
                self.guidence_label,
                self.guidence_input,
                self.seed_label,
                self.seed_input,
                self.count_label,
                self.count_input,
            ],
        ]

        max_opts = max([len(opt) for opt in opts])
        px = max_opts
        cur = 0

        # add widgets to layout
        grid.addWidget(self.image_label, 0, 0, px, px, Qt.AlignmentFlag.AlignCenter)
        cur += px

        grid.addWidget(self.progress_bar, cur, 0, 1, px)
        cur += 1

        grid.addWidget(self.prompt_input, cur, 0, 1, px)
        cur += 1

        grid.addWidget(self.generate_button, cur, 0, 1, px)
        cur += 1

        for i, opt in enumerate(opts):
            for j, widget in enumerate(opt):
                grid.addWidget(widget, i + cur, j, 1, 1)
        cur += 1

        self.inited = False
        self.setFixedWidth(pixmap.width())

        self.is_generating = False
    
    def set_max_progress(self, max_progress):
        self.max_progress = max_progress
        self.progress_bar.setRange(0, max_progress)
    
    def initial_center(self):
        self.center()
    
    def center(self):
        qr = self.frameGeometry()
        cp = self.screen().availableGeometry().center()

        qr.moveCenter(cp)
        self.move(qr.topLeft())

    def init(self):
        if self.inited:
            return
        self.inited = True

        self.center()

        self.setMaximumSize(QWIDGETSIZE_MAX, QWIDGETSIZE_MAX)
        self.setMinimumSize(0, 0)

    def resizeEvent(self, event):
        if not self.inited:
            self.init()
    
    def enter_generating_mode(self):
        if self.is_generating:
            return False

        self.is_generating = True
        # self.generate_button.setEnabled(False)
        return True
    
    def exit_generating_mode(self):
        self.is_generating = False
        # self.generate_button.setEnabled(True)
    
    def button_clicked(self):
        if not self.enter_generating_mode():
            return

        prompt = self.prompt_input.toPlainText()
        steps = int(self.steps_input.text())
        guidence = float(self.guidence_input.text())
        seed = int(self.seed_input.text())
        count = int(self.count_input.text())

        if seed == -1:
            seed = randint(0, 2**32 - 1)
        
        self.ui.logger.info(f"Prompt: {prompt}, guidence: {guidence}, seed: {seed}, count: {count}")
        Thread(target=self.generate_thread, args=(prompt, steps, guidence, seed, count)).start()
    
    def create_generation_step_callback(self):
        def callback(i, t, latents):
            image = self.ui.model.decode_latents(latents)[0]

            pixmap = np_to_pixmap(image)
            self.image_label.setPixmap(pixmap)

            # self.progress_bar.setValue(i)

        return callback
    
    def generate(self, prompt, steps, guidence, seed):
        self.ui.logger.info(f"Prompt: {prompt}, guidence: {guidence}, seed: {seed}")
        image = self.ui.model(
            prompt=prompt,
            num_inference_steps=steps,
            guidance_scale=guidence,
            seed=seed,
            callback=self.create_generation_step_callback()
        )
    
    def generate_thread(self, prompt, steps, guidence, seed, count):
        for i in range(count):
            self.set_max_progress(steps)
            self.generate(prompt, steps, guidence, seed)
        self.exit_generating_mode()


In [8]:
class UI:
    app = QApplication([])

    def __init__(self, model, logger) -> None:
        self.window = MainWindow(self)
        self.window.show()

        self.model = model
        self.logger = logger
        
    def exec(self):
        UI.app.exec()

In [9]:
ui = UI(model, logger)

In [10]:
ui.exec()

INFO:python_coreml_stable_diffusion.pipeline:Prompt: cat, guidence: 7.5, seed: 682539326, count: 1
INFO:python_coreml_stable_diffusion.pipeline:Prompt: cat, guidence: 7.5, seed: 682539326
INFO:python_coreml_stable_diffusion.pipeline:Setting random seed to 682539326
  0%|          | 0/11 [00:00<?, ?it/s]

: 

: 